# Creating monthly balance in Python from SQL script

## Setup

It is needed to import neccesary libraries, `sqlite3` will help taking the SQL script to python and `pandas` will be use for a best look and fell in the data preview

In [1]:
import sqlite3
import pandas as pd

Just to get a reference for the current directory, can help adjusting the `path` in next steps

In [2]:
import os
os.getcwd()

'C:\\Users\\Home\\Documents\\GitHub\\Analytics-Case\\scripts'

Stablishing connection with the database and creating the cursor. It will be neccesary to adjust the path, according to user's database allocation.  

In [3]:
path = 'C:\\Users\\Home\\Documents\\GitHub\\Analytics-Case\\data\\database\\db'
sqliteConnection = sqlite3.connect(path)
cursor =  sqliteConnection.cursor()

Double-checking connection, it should gives the provided tables: [('country',), ('state',), ('city',), ('customers',), ('accounts',), ('transfer_ins',), ('transfer_outs',), ('pix_movements',), ('d_year',), ('d_week',), ('d_weekday',), ('d_month',), ('d_time',), ('Balance',)]

In [4]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('country',), ('state',), ('city',), ('customers',), ('accounts',), ('transfer_ins',), ('transfer_outs',), ('pix_movements',), ('d_year',), ('d_week',), ('d_weekday',), ('d_month',), ('d_time',), ('Balance',)]


## Running queries 

Using SQL queries to generate the expected results

In [5]:
query_drop = 'DROP TABLE IF EXISTS Balance;'

query_balance = '''
  CREATE TABLE Balance (
  month INT,
  customer_id UUID,
  first_name VARCHAR(128),
  last_name VARCHAR(128),
  transfer_in FLOAT,
  transfer_out FLOAT
);
        '''
query_ti = '''
INSERT INTO Balance (month, customer_id, first_name, last_name, transfer_in, transfer_out)
SELECT dm.action_month, a.customer_id, c.first_name, c.last_name, ti.amount, 0 
FROM transfer_ins ti
JOIN accounts a ON ti.account_id = a.account_id
JOIN customers c ON a.customer_id = c.customer_id
JOIN d_time dt ON ti.transaction_completed_at = dt.time_id
JOIN d_month dm on dt.month_id = dm.month_id
JOIN d_year dy on dt.year_id = dy.year_id
WHERE ti.status = 'completed' and dy.action_year='2020';
'''

query_to = '''
INSERT INTO Balance (month, customer_id, first_name, last_name, transfer_in, transfer_out)
SELECT dm.action_month, a.customer_id, c.first_name, c.last_name, 0 , tou.amount
FROM transfer_outs tou
JOIN accounts a ON tou.account_id = a.account_id
JOIN customers c ON a.customer_id = c.customer_id
JOIN d_time dt ON tou.transaction_completed_at = dt.time_id
JOIN d_month dm on dt.month_id = dm.month_id
JOIN d_year dy on dt.year_id = dy.year_id
WHERE tou.status = 'completed' and dy.action_year='2020';
'''

query_pix = '''
INSERT INTO Balance (month, customer_id, first_name, last_name, transfer_in, transfer_out)
SELECT dm.action_month, a.customer_id, c.first_name, c.last_name, CASE WHEN in_or_out like '%in%' THEN pix_amount ELSE 0 END, CASE WHEN in_or_out like '%out%' THEN pix_amount ELSE 0 END 
FROM pix_movements pix
JOIN accounts a ON pix.account_id = a.account_id
JOIN customers c ON a.customer_id = c.customer_id
JOIN d_time dt ON pix.pix_completed_at = dt.time_id
JOIN d_month dm on dt.month_id = dm.month_id
JOIN d_year dy on dt.year_id = dy.year_id
WHERE pix.status = 'completed' and dy.action_year='2020';
'''

query_final = '''
SELECT month, customer_id, first_name, last_name, SUM(transfer_in) as 'Total Transfer In', SUM(transfer_out) as 'Total Transfer out', SUM(SUM(transfer_in)-SUM(transfer_out)) OVER (PARTITION BY customer_id ORDER BY month) AS 'Account Monthly Balance' 
FROM Balance
GROUP BY customer_id, month
ORDER BY month;
'''

cursor.execute(query_drop)
cursor.execute(query_balance)
cursor.execute(query_ti)
cursor.execute(query_to)
cursor.execute(query_pix)
pd.read_sql_query(query_final, sqliteConnection)


,month,customer_id,first_name,last_name,Total Transfer In,Total Transfer out,Account Monthly Balance
0,1,463741281156103,Mark,Sawyer,3095.85,224.12,2871.73
1,1,1320946992314442,Margareta,Haupt,8516.01,5204.96,3311.05
2,1,1671564189319424,Catherine,Laman,3618.70,3587.88,30.82
3,1,2358908589613037,Linda,Eargle,11198.11,5440.35,5757.76
4,1,4055589654669036,Jackie,Kahele,6636.44,4342.56,2293.88
...,...,...,...,...,...,...,...
45552,12,3399368655915363840,Caroline,Ciprian,195.89,1962.91,18798.83
45553,12,3399413968938289152,Virginia,Blake,4097.53,3901.42,7758.26
45554,12,3400407496661060096,Bessie,Brown,4086.24,4299.85,22079.03
45555,12,3401963777864371200,Nicole,Hughes,4206.36,8455.94,-19524.25


## Closing cursor and connection 

In [6]:
cursor.close()
sqliteConnection.close()